<a href="https://colab.research.google.com/github/AllergicAlligator/skyguard/blob/main/Ocean_Challenge_2024_Skyguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip model.zip

Archive:  model.zip
 extracting: keras_model.h5          
 extracting: labels.txt              


In [ ]:
!pip install gradio

In [ ]:
!curl -o linenotify.py https://raw.githubusercontent.com/AllergicAlligator/skyguard/main/linenotify.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   818  100   818    0     0   3837      0 --:--:-- --:--:-- --:--:--  3840


In [ ]:
import gradio as gr

In [ ]:
from linenotify import Notify

In [ ]:
token = ""

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

def process(image):
  # Disable scientific notation for clarity
  np.set_printoptions(suppress=True)

  # Load the model
  model = load_model("keras_model.h5", compile=False)

  # Load the labels
  class_names = open("labels.txt", "r").readlines()

  # Create the array of the right shape to feed into the keras model
  # The 'length' or number of images you can put into the array is
  # determined by the first position in the shape tuple, in this case 1
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

  # Replace this with the path to your image
  # image = Image.open("<IMAGE_PATH>").convert("RGB")

  # resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

  # turn the image into a numpy array
  image_array = np.asarray(image)

  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

  # Load the image into the array
  data[0] = normalized_image_array

  # Predicts the model
  prediction = model.predict(data)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]

  # Print prediction and confidence score
  # print("Class:", class_name[2:], end="")
  # print("Confidence Score:", confidence_score)

  image.save("test.jpg")
  Notify(token, msg=f"{class_name[2:]}\n{confidence_score}", img="test.jpg")

  return class_name[2:], confidence_score


In [ ]:
gr.Interface(process,
             gr.Image(type="pil"),
             [gr.Text(label="Class"), gr.Text(label="Confidence")],
             title="Garbage Classification"
             ).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e068dfcf46fa6d02e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
